In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('finger-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')



anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))


rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))




def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

def sizeT():
    return relT(tN) 
day=list(range(day_time(t0),day_time(tN)+1))[0]

from datetime import datetime, timedelta


def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")




day_str=getStrDatefrom(day)
print(day_str)
users=config.get('Configuration', day_str)
users = [(item.strip()) for item in users.split(',')]


print(users)


2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59
2024-07-31 02:00:00
2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
2024-07-31
['16fe', '5b66', 'ed9c']


In [3]:
import os

df_total_map = {}
for user in users:
    df_total_map[user]=None
for user in users:
    print(os.path.join(source_folder,"ubitrack-uwb")+"/"+str(day)+"."+user+".sourcedata.tsv")
    try:
        series=read_csv(os.path.join(source_folder,"ubitrack-uwb")+"/"+str(day)+"."+user+".sourcedata.tsv", sep="\t", header=None, names=["time","date","anchor","d"])
        print(day,user,series.size)
        series['time'] = pd.to_numeric(series['time'], errors='coerce')
        series = series.dropna(subset=['time'])
        #print("unique",np.unique(series["anchor"]))
    except Exception as e:
        print("Not data",e)
        continue
    if(df_total_map[user] is None):
        df_total_map[user]=series
    else:
        df_total_map[user] = df_total_map[user].append(series, ignore_index=True)


for user in users:
    if(df_total_map[user] is not None):
        df_total_map[user] = df_total_map[user].sort_values(by='time', ascending=True)
    print(df_total_map[user])


./raw-har/ubitrack-uwb/19935.16fe.sourcedata.tsv
19935 16fe 1051644
./raw-har/ubitrack-uwb/19935.5b66.sourcedata.tsv


19935 5b66 1117276
./raw-har/ubitrack-uwb/19935.ed9c.sourcedata.tsv
19935 ed9c 901328
              time                 date anchor      d
0       1722384000  2024-07-31 02:00:00   399a  6.046
1       1722384000  2024-07-31 02:00:00   6aca  2.678
2       1722384000  2024-07-31 02:00:00   45d2  5.512
3       1722384001  2024-07-31 02:00:01   399a  6.060
4       1722384001  2024-07-31 02:00:01   6aca  2.678
...            ...                  ...    ...    ...
262906  1722470396  2024-08-01 01:59:56   2eff  5.719
262907  1722470397  2024-08-01 01:59:57   6aca  2.215
262908  1722470398  2024-08-01 01:59:58   6aca  2.224
262909  1722470399  2024-08-01 01:59:59   2eff  6.173
262910  1722470399  2024-08-01 01:59:59   6aca  2.224

[262911 rows x 4 columns]
              time                 date anchor      d
0       1722384000  2024-07-31 02:00:00   8171  4.120
1       1722384000  2024-07-31 02:00:00   a304  3.713
2       1722384000  2024-07-31 02:00:00   eb38  4.079
3       1722384001  202

In [4]:
df_total_map2 = {}

for user in users:
    print(user)
    df_total_map2[user]={}
    df_total=df_total_map[user]
    for anchor in anchors:
        print(anchor)
        df_total_map2[user][anchor]=np.ones((sizeT(),2))
        df_total_map2[user][anchor][:,0]=df_total_map2[user][anchor][:,0]*10
        df_total_map2[user][anchor][:,1]=df_total_map2[user][anchor][:,1]*-10
        if(df_total is None):
            continue
        lastVmin,lastVmax=10,-10
        i0=0
        for index, row in df_total.iterrows():
            if(row["anchor"]!=anchor):
                continue
            iN=relT(int(row["time"]))
            if(iN<0):
                iN=0
            if(iN>=relT(tN)):
                iN=relT(tN)-1
            #print(iN,i0)
            
            if(i0!=iN):
                #print("\t",user,anchor,i0,iN,(lastVmin,lastVmax), row["d"])
                for ii in range(i0,iN-1,1):
                    df_total_map2[user][anchor][ii][0]=10
                    df_total_map2[user][anchor][ii][1]=-10
                df_total_map2[user][anchor][i0][0]=lastVmin
                df_total_map2[user][anchor][i0][1]=lastVmax
                i0=iN
                lastVmin,lastVmax=row["d"],row["d"]
            else:
                lastVmin=min(lastVmin,row["d"])
                lastVmax=max(lastVmax,row["d"])
        print(df_total_map2[user][anchor])

        


16fe
1fa0


[[ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]
 ...
 [ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]]
2eff


[[ 10.    -10.   ]
 [ 10.    -10.   ]
 [ 10.    -10.   ]
 ...
 [  5.719   5.719]
 [ 10.    -10.   ]
 [ 10.    -10.   ]]
399a


[[  6.046   6.046]
 [  6.06    6.06 ]
 [  6.037   6.037]
 ...
 [ 10.    -10.   ]
 [ 10.    -10.   ]
 [ 10.    -10.   ]]
45d2


[[  5.512   5.512]
 [  5.672   5.672]
 [  5.573   5.573]
 ...
 [ 10.    -10.   ]
 [ 10.    -10.   ]
 [ 10.    -10.   ]]
6aca


...

[[ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]
 ...
 [ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]]
8171


[[ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]
 ...
 [ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]]
a304


[[ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]
 ...
 [ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]]
c492


[[  4.149   4.149]
 [  4.093   4.13 ]
 [  4.088   4.107]
 ...
 [  3.647   3.647]
 [  3.606   3.606]
 [ 10.    -10.   ]]
eb38


[[ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]
 ...
 [ 10. -10.]
 [ 10. -10.]
 [ 10. -10.]]


In [5]:
import cv2
import os

os.makedirs(data_folder+"/DAY_"+str(day)+"/", exist_ok=True)

for user in users:
    print(user)
    for anchor in anchors:
        print(anchor)
        print(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
        filew = open(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", "w")
   
        for tx,tt in enumerate(range(t0,tN,time_step)):
            #print(tt,time2str(tt),anchor)
            #if(day_time(tt) != day):
            #    continue

#            min_v=10
#            max_v=-10

#            df_total=df_total_map[user]
            #print(df_total)
#            df_intervalo = df_total[(df_total['time'] >= (tt-1)) & (df_total['time'] <= (tt)) & (df_total['anchor'] ==anchor) ]
#            if(df_intervalo.size>0):
#                min_v=min(min_v,np.min(df_intervalo['d']))
#                max_v=max(max_v,np.max(df_intervalo['d']))

            filew.write(str(tt)+"\t"+str(df_total_map2[user][anchor][tx][0])+"\t"+str(df_total_map2[user][anchor][tx][1])+"\n")
        filew.close()


16fe
1fa0
./data//DAY_19935/16fe.1fa0.tsv
2eff
./data//DAY_19935/16fe.2eff.tsv


399a
./data//DAY_19935/16fe.399a.tsv
45d2
./data//DAY_19935/16fe.45d2.tsv


6aca
./data//DAY_19935/16fe.6aca.tsv
8171
./data//DAY_19935/16fe.8171.tsv


a304
./data//DAY_19935/16fe.a304.tsv
c492
./data//DAY_19935/16fe.c492.tsv


eb38
./data//DAY_19935/16fe.eb38.tsv
5b66
1fa0
./data//DAY_19935/5b66.1fa0.tsv


...

ed9c
1fa0
./data//DAY_19935/ed9c.1fa0.tsv
2eff
./data//DAY_19935/ed9c.2eff.tsv


399a
./data//DAY_19935/ed9c.399a.tsv
45d2
./data//DAY_19935/ed9c.45d2.tsv


6aca
./data//DAY_19935/ed9c.6aca.tsv
8171
./data//DAY_19935/ed9c.8171.tsv


a304
./data//DAY_19935/ed9c.a304.tsv
c492
./data//DAY_19935/ed9c.c492.tsv


eb38
./data//DAY_19935/ed9c.eb38.tsv
